In [ ]:
У додатку ви знайдете файл, котрий містить дані та зображення - візуалізацію на основі цих даних

Ваша задача:
- Ідентифікувати та описати проблеми у цій візуалізації даних (я очікую від вас текст)
- Зробити альтернативний варіант презентації цих даних (я очікую від вас візуалізацію та пояснення, чому вона працює ліпше, ніж оригінальна)

Поточна візуалізація:
![old image](old.png "Old_image")

Перелік помилок, які я побачив:
1) зображуються різні види даних (регіони, країна загалом)<br/>
2) мало того, що дані в ряд, так ще й кожен стовпчик щось показує<br/>
3) вертикальна шкала тільки заплутує орієнтацію по наповненності колонки<br/>
4) текст горизонтальної шкали повернутий і складно читається<br/>
5) сама картинка трохи сплюснута по вертикалі (по тексту це видно)<br/>
6) я б ще "послуги" якось скоротив, бо дуже велика назва

In [80]:
import sys
!{sys.executable} -m pip install pandas altair

You should consider upgrading via the '/Users/mykyta/aastudy/visual/venv/bin/python3 -m pip install --upgrade pip' command.


In [81]:
import pandas as pd
import altair as alt
import datetime as dt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [82]:
def load_data():
    return pd.read_excel('1.4_02.xlsx')

df = load_data()
df.head()

,"Регіон, рік",сільське господарство,промисловість,будівництво,послуги,державне управління
0,"Україна, 2010",8.3,25.9,3.3,57.3,5.2
1,"Україна, 2017",12.1,25.2,2.5,53.7,6.5
2,"Вінницька, 2010",21.6,19.3,4.1,47.4,7.6
3,"Вінницька, 2017",28.1,21.4,2.0,39.8,8.7
4,"Волинська, 2010",16.9,15.6,3.2,57.7,6.6


In [83]:
# Transforming data

# instead of having multiple different columns, we will make 'category' column.
# Also separate year and region
# And add colors

transformed_df = {'year': [], 'region': [], 'value': [], 'category': []}
for a, b in df.iterrows():
    reg, year = b['Регіон, рік'].split(',')
    year = year.strip()
    temp_dict = {'сільгосп': b['сільське господарство'],
                 'промисловість': b['промисловість'],
                 'будівництво': b['будівництво'],
                 'послуги': b['послуги'],
                 'держуправління': b['державне управління']}
    for category, value in temp_dict.items():
        transformed_df['year'].append(year)
        transformed_df['region'].append(reg)
        transformed_df['value'].append(value)
        transformed_df['category'].append(category)

transformed_df = pd.DataFrame(transformed_df)

In [84]:
transformed_df.head()

,year,region,value,category
0,2010,Україна,8.3,сільгосп
1,2010,Україна,25.9,промисловість
2,2010,Україна,3.3,будівництво
3,2010,Україна,57.3,послуги
4,2010,Україна,5.2,держуправління


In [85]:
# next thing we will separate Ukraine from other regions, since it is other data
ukr_only = transformed_df[:10]
transformed_df = transformed_df[11:]

In [86]:
ukr_only.head()

,year,region,value,category
0,2010,Україна,8.3,сільгосп
1,2010,Україна,25.9,промисловість
2,2010,Україна,3.3,будівництво
3,2010,Україна,57.3,послуги
4,2010,Україна,5.2,держуправління


In [87]:
transformed_df.head()

,year,region,value,category
11,2010,Вінницька,19.3,промисловість
12,2010,Вінницька,4.1,будівництво
13,2010,Вінницька,47.4,послуги
14,2010,Вінницька,7.6,держуправління
15,2017,Вінницька,28.1,сільгосп


In [88]:
input_dropdown = alt.binding_select(options = ukr_only.category.unique())
select_category = alt.selection_single(fields = ['category'], bind = input_dropdown)

In [89]:
dom = ['2010', '2017']
rng = ['blue', 'orange']

ukr_chart = alt.Chart(ukr_only).mark_bar().encode(
    y = alt.Y('value:Q'),
    x = alt.X('year:N'),
    color=alt.condition(
        alt.datum.year == '2010',
        alt.value('#849cc4'),
        alt.value('#f1aa5e'),
    )
).add_selection(
    select_category
).transform_filter(select_category)

In [90]:
lines = alt.Chart(transformed_df).mark_line().encode(
    x = alt.X('max(value):Q'),
    x2 = alt.X2('min(value):Q'),
    y = alt.Y('region:N', sort=alt.Sort(field='region', op='max', order='descending')))

points = alt.Chart(transformed_df).mark_circle(size = 100).encode(
    x = alt.X('value:Q'),
    y = alt.Y('region:N', sort=alt.Sort(field='region', op='max', order='descending')),
    color = alt.Color('year:N')
)

Result:
Чому моя візуалізація краще:
1) Всі типи даних однакові. Окремо регіони, окремо (поруч справа) країна загалом<br/>
2) Показується одна категорія даних на дисплеї за раз. Немає перенасичення даними на картинці<br/>
3) Вертикальна шкала допомагає зрозуміти про який регіон говориться, горизонтальна +- чітко показує значення<br/>
4) На основній таблиці текст горизонтальний<br/>
5) картинка не сплюснута і навіть інтерактивна
![new image](new.gif "New_visualisation")

In [91]:
alt.hconcat(
    alt.layer(lines, points)
    .add_selection(select_category)
    .transform_filter(select_category)
    .properties(width = 600, height = 600, title="Зміна структури ВДВ у розрізі регіонів за 2017 рік порівняно з 2010 роком"),
    ukr_chart.properties(width=100, height = 600)
)

alt.HConcatChart(...)